In [ ]:
!pip install -q langchain langchain_community langchain_groq langchain_cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
prompt = PromptTemplate.from_template(
    """
    You are given a list of company names, find the stock symbol(s) for them.
    You have access to the search tool
    tools: {tools}\n
    tool name: {tool_name} \n
    You can use the tool if you do not know the symbol, if you know then there is no need to search
    Return list of symbols \n
    input: company names {company}
    """
)

In [ ]:
from google.colab import userdata
import os

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_KEY')

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [ ]:
tools[0].metadata

In [ ]:
from typing import List

class Symbols(BaseModel):
  "list of stock market symbols"
  symbols: List[str] = Field(description="list of stock market symbols for the input companies")

In [ ]:
agent = ChatGroq(model="llama3-70b-8192", temperature=0, streaming=True, verbose=False)
agent_struct = agent.with_structured_output(Symbols)

In [ ]:
agent_chain = prompt | agent

In [ ]:
agent.invoke("heloo")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'finish_reason': 'stop'}, id='run-697628b7-7e42-403a-b2e8-354bd69fbd35-0')

In [ ]:
type(tools[0])

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent

def create_agent(llm, tools: list):
  # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are given a list of company names, find the stock symbol(s) for them.
                   You have access to the search tool tavily_search_results_json
                   You can use the tool if you do not know the symbol, if you know then there is no need to search
                   input: list of company names
                   output: Return only the symbols separated by spaces. Don't add any type of punctuantion.
                """,
            ),
           ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [ ]:
agent_chain = create_agent(agent, tools)

In [ ]:
agent_chain.invoke("Apple Tesla"})

AttributeError: 'Stream' object has no attribute 'dict'

In [ ]:
prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are given a list of company names, find the stock symbol(s) for them.
                   You have access to the search tool tavily_search_results_json
                   You can use the tool if you do not know the symbol, if you know then there is no need to search
                   Return list of stock symbols corresponding to the company\n
                   input: list of company names
                """,
            ),
            MessagesPlaceholder(variable_name="input"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

In [ ]:
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_KEY')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere

model = ChatCohere(model='command-r-plus', streaming=True)

prompt = ChatPromptTemplate.from_template(
    "You are and finance expert AI Assistant. \n"
    "Your are provided a json string consisting of meaningful data about {topic} of "
    "{symbol}."
    "Your task is to create a meaningful report using it, so that the report can be used by a supervisor as context "
    "for final decision.\n"
    "json string: {json}"
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
av_key = userdata.get('AV_KEY')

In [ ]:
from datetime import date, timedelta
from datetime import datetime
import requests
import json

def get_news_search(symbol):
    """
        get outsider sentiment
        get news with sentiment score about company with stock symbol passed as input. \n
        news will be used for sentiment analysis for finance & stock trade decisions
        output: json containing the metadata and news
        """
    now = datetime.now() - timedelta(days=30)
    formatted_time = now.strftime("%Y%m%dT%H%M")
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&time_from={formatted_time}&tickers={symbol}&sort=LATEST&apikey={av_key}'
    r = requests.get(url)
    data = r.json()

    return json.dumps(data)

In [ ]:
info = get_news_search("TSLA")

In [ ]:
info

'{"items": "50", "sentiment_score_definition": "x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish", "relevance_score_definition": "0 < x <= 1, with a higher score indicating higher relevance.", "feed": [{"title": "World\'s Smartest Banker Says \\"Could Be Hell To Pay\\" If Wall Street\'s Craze Goes Wrong - iShares 20+ Year Treasury Bond ETF  ( NASDAQ:TLT ) ", "url": "https://www.benzinga.com/markets/24/05/39092931/worlds-smartest-banker-says-could-be-hell-to-pay-if-wall-streets-craze-goes-wrong", "time_published": "20240530T165243", "authors": ["The Arora Report"], "summary": "To gain an edge, this is what you need to know today. Please click here for an enlarged chart of iShares 20+ Year Treasury Bond ETF TLT. High yield: 4.650% ( When-Issued: 4.637% ) GDP - second estimate for Q1 came at 1.3% vs. 1.3% consensus. You may recall that the advance report showed a ...", "banner_image": "https://cdn.

In [ ]:
chain.invoke({"topic": "company news with sentiment score", "symbol": "TSLA", "json": info})

"Here is a report on Tesla (TSLA) based on the provided JSON data: \n\n## Summary: \nThe report provides an analysis of recent news and sentiment surrounding Tesla, Inc. (TSLA) as of May 30, 2024. The data includes articles from various sources, such as Benzinga, Motley Fool, Business Insider, and more. Overall, the sentiment towards TSLA is mostly neutral, with a few articles leaning slightly bullish or bearish. The topics discussed include Tesla's pricing strategies, Elon Musk's compensation package, and the development of electric vehicles (EVs) by legacy automakers. \n\n## Details: \n- **Pricing Strategies**: There is discussion about Tesla's strategy of lowering prices, which is great for car buyers but may not be beneficial for the company's profits. \n- **Elon Musk's Compensation**: There are differing opinions on Elon Musk's 2018 compensation package, with some investors and experts expressing support while others plan to vote against it. \n- **Electric Vehicle Competition**: L

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent_tools = [TavilySearchResults(max_results=3)]

agent_llm = ChatGroq(model="llama3-70b-8192", temperature=0, streaming=True)


def create_agent(llm, tools: list):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are given a list of company names, find the stock symbol(s) for them.
                   You have access to the search tool tavily_search_results_json
                   You can use the tool if you do not know the symbol, if you know then there is no need to search
                   input: list of company names
                   output: Return only the symbols separated by spaces. Don't add any type of punctuantion.
                """,
            ),
            MessagesPlaceholder(variable_name="input"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


agent_chain = create_agent(agent_llm, agent_tools)

In [ ]:
from langchain_core.messages import HumanMessage

await agent_chain.ainvoke({"input": [HumanMessage(content="Tesla")]})

AttributeError: 'AsyncStream' object has no attribute 'dict'

In [ ]:
!pip install yahooquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
from yahooquery import Ticker

In [ ]:
def get_fin_statements(symbol):
    df = Ticker(symbol)
    df1 = df.income_statement().reset_index(drop=True).transpose()
    df2 = df.balance_sheet().reset_index(drop=True).transpose()
    df3 = df.cash_flow().reset_index(drop=True).transpose()
    return df1, df2, df3

In [ ]:
income_stmt, balance_sheet, cash_flow = get_fin_statements('TSLA')

In [ ]:
income_stmt.to_csv('income_stmt.csv')


balance_sheet.to_csv('balance_sheet.csv')


cash_flow.to_csv('cash_flow.csv')

In [ ]:
!pip install -q langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 32.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=987a6b509d21be6f7d8837f87327e53af7a233fa95bbaaf27994fd565ee1e500
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
item

[Document(page_content='\n\n\n\n0\n1\n2\n3\n4\n5\n6\n\n\nasOfDate\n2020-12-31 00:00:00\n2021-12-31 00:00:00\n2022-12-31 00:00:00\n2023-09-30 00:00:00\n2023-12-31 00:00:00\n2023-12-31 00:00:00\n2024-03-31 00:00:00\n\n\nperiodType\n12M\n12M\n12M\nTTM\n12M\nTTM\nTTM\n\n\ncurrencyCode\nUSD\nUSD\nUSD\nUSD\nUSD\nUSD\nUSD\n\n\nAverageDilutionEarnings\n\n9000000.0\n1000000.0\n\n0.0\n\n\n\n\nBasicAverageShares\n2799000000.0\n2959000000.0\n3130000000.0\n3168250000.0\n3173500000.0\n3174000000.0\n3179000000.0\n\n\nBasicEPS\n0.246667\n1.87\n4.02\n3.41\n4.725697\n4.73\n4.3\n\n\nCostOfRevenue\n24906000000.0\n40217000000.0\n60609000000.0\n\n79113000000.0\n\n77900000000.0\n\n\nDilutedAverageShares\n3249000000.0\n3386000000.0\n3475000000.0\n3480250000.0\n3482750000.0\n3485000000.0\n3489000000.0\n\n\nDilutedEPS\n0.213333\n1.63\n3.62\n3.1\n4.31\n4.3\n3.91\n\n\nDilutedNIAvailtoComStockholders\n721000000.0\n5533000000.0\n12584000000.0\n\n14999000000.0\n\n13652000000.0\n\n\nEBIT\n1902000000.0\n6714000000.0\n

In [ ]:
item[0].page_content

'\n\n\n\n0\n1\n2\n3\n4\n5\n6\n\n\nasOfDate\n2020-12-31 00:00:00\n2021-12-31 00:00:00\n2022-12-31 00:00:00\n2023-09-30 00:00:00\n2023-12-31 00:00:00\n2023-12-31 00:00:00\n2024-03-31 00:00:00\n\n\nperiodType\n12M\n12M\n12M\nTTM\n12M\nTTM\nTTM\n\n\ncurrencyCode\nUSD\nUSD\nUSD\nUSD\nUSD\nUSD\nUSD\n\n\nAverageDilutionEarnings\n\n9000000.0\n1000000.0\n\n0.0\n\n\n\n\nBasicAverageShares\n2799000000.0\n2959000000.0\n3130000000.0\n3168250000.0\n3173500000.0\n3174000000.0\n3179000000.0\n\n\nBasicEPS\n0.246667\n1.87\n4.02\n3.41\n4.725697\n4.73\n4.3\n\n\nCostOfRevenue\n24906000000.0\n40217000000.0\n60609000000.0\n\n79113000000.0\n\n77900000000.0\n\n\nDilutedAverageShares\n3249000000.0\n3386000000.0\n3475000000.0\n3480250000.0\n3482750000.0\n3485000000.0\n3489000000.0\n\n\nDilutedEPS\n0.213333\n1.63\n3.62\n3.1\n4.31\n4.3\n3.91\n\n\nDilutedNIAvailtoComStockholders\n721000000.0\n5533000000.0\n12584000000.0\n\n14999000000.0\n\n13652000000.0\n\n\nEBIT\n1902000000.0\n6714000000.0\n13910000000.0\n\n101290